In [ ]:
# SHARED UTILITIES

from pathlib import Path
import ast
from sage.all import *

In [ ]:

# Global caches for performance
_matrix_cache = {}
_reduction_cache = {}
_primitive_vector_cache = {}
_orbit_cache = {}
_stabilizer_cache = {}


In [ ]:
def _R(n):
    """Create integer ring mod n."""
    return Integers(Integer(n))

def _lift_int(x):
    """Lift integer from ring to Python int."""
    if hasattr(x, "lift"):
        return Integer(x.lift())
    return Integer(x)

def _flat_entries(M):
    """Flatten a 2x2 matrix-like object to [a,b,c,d]."""
    Mm = M.matrix() if hasattr(M, "matrix") else M
    L = Mm.list()
    if len(L) == 2 and isinstance(L[0], (list, tuple)):
        L = list(L[0]) + list(L[1])
    if len(L) != 4:
        L = [Mm[0,0], Mm[0,1], Mm[1,0], Mm[1,1]]
    return L


In [ ]:
def _mat_key(M):
    """Hash key for a matrix over Z/nZ."""
    L = _flat_entries(M)
    return tuple(ZZ(x) for x in L)

def _vec_key(v):
    """Hash key for a vector in (Z/nZ)^2."""
    return (ZZ(v[0]), ZZ(v[1]))

def _to_mat_over_R(g, R):
    """Convert generator to matrix over ring R."""
    if hasattr(g, "matrix"):
        g = g.matrix()
    if hasattr(g, "nrows"):
        return Matrix(R, 2, 2, [ _lift_int(e) for e in g.list() ])
    if isinstance(g, (list, tuple)):
        if len(g) == 4:
            return Matrix(R, 2, 2, [ _lift_int(e) for e in g ])
        if len(g) == 2 and all(isinstance(row, (list, tuple)) and len(row) == 2 for row in g):
            return Matrix(R, 2, 2, [ _lift_int(g[0][0]), _lift_int(g[0][1]),
                                     _lift_int(g[1][0]), _lift_int(g[1][1]) ])
    raise TypeError("Unrecognized generator format.")


In [ ]:
def _reduce_mod_cached(M, n):
    """Cached matrix reduction."""
    key = (_mat_key(M), n)
    if key not in _reduction_cache:
        n = Integer(n)
        Rn = _R(n)
        L = _flat_entries(M)
        _reduction_cache[key] = matrix(Rn, 2, 2, [Rn(ZZ(x)) for x in L])
    return _reduction_cache[key]

def _unique_reductions(elems, n):
    """Unique reductions with caching."""
    seen = {}
    for g in elems:
        rg = _reduce_mod_cached(g, n)
        seen[_mat_key(rg)] = rg
    return list(seen.values())


In [ ]:
def primitive_vectors_exact_order(n):
    """Primitive vector computation with caching."""
    n = Integer(n)
    if n in _primitive_vector_cache:
        return _primitive_vector_cache[n]
    
    Rn = _R(n)
    out = []
    # Use more efficient iteration with early gcd checks
    for x in range(int(n)):
        if gcd(n, x) != 1:  # Skip if x shares factor with n
            continue
        for y in range(int(n)):
            if gcd(x, y) == 1:  # Only need to check x,y now
                out.append(vector(Rn, [x, y]))
    
    _primitive_vector_cache[n] = out
    return out


In [ ]:
def orbit_reps_on_vectors(vlist, gens):
    """Orbit representatives with early exits and caching."""
    if not vlist:
        return []
    
    # Create cache key for generator set
    gen_key = tuple(_mat_key(g) for g in gens)
    cache_key = (gen_key, tuple(_vec_key(v) for v in vlist))
    
    if cache_key in _orbit_cache:
        return _orbit_cache[cache_key]

    gens2 = list(gens) + [g.inverse() for g in gens]
    remaining = {_vec_key(v): v for v in vlist}
    reps = []

    while remaining:
        k0, v0 = next(iter(remaining.items()))
        reps.append(v0)

        orb = set([k0])
        queue = [v0]
        while queue:
            w = queue.pop()
            for g in gens2:
                w2 = g * w
                k2 = _vec_key(w2)
                if k2 not in orb:
                    orb.add(k2)
                    queue.append(w2)

        for k in orb:
            remaining.pop(k, None)

    _orbit_cache[cache_key] = reps
    return reps


In [ ]:
def stabilizer_elements_from_element_list(G_elems, v):
    """Stabilizer computation with caching."""
    v_key = _vec_key(v)
    elems_key = tuple(_mat_key(g) for g in G_elems)
    cache_key = (elems_key, v_key)
    
    if cache_key in _stabilizer_cache:
        return _stabilizer_cache[cache_key]
    
    stab = []
    for g in G_elems:
        if g * v == v:
            stab.append(g)
    
    _stabilizer_cache[cache_key] = stab
    return stab

def generated_subgroup_order(mats):
    """Subgroup order computation."""
    if not mats:
        return 1
    return MatrixGroup(mats).order()


In [ ]:
def parse_count_file(path):
    """File parser with minimal field extraction."""
    wanted = {
        "label": "label",
        "adelic-gens": "adelic_gens",
        "adelic-level": "N",
        "m-level": "m0",
        "count": "count",
        "m0-bound": "m0_bound",
        "index-bound": "index_bound",
    }

    blocks = []
    cur = {}
    in_prim = False

    with open(path, "r", encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()

            if line == "":
                if cur:
                    blocks.append(cur)
                    cur = {}
                in_prim = False
                continue

            if line.startswith("primitive-point="):
                in_prim = True
                continue
            if in_prim:
                if line == "*}" or line.endswith("*}"):
                    in_prim = False
                continue

            if "=" not in line:
                continue

            k, v = line.split("=", 1)
            k = k.strip()
            v = v.strip()

            if k not in wanted:
                continue

            outk = wanted[k]
            if k == "label":
                cur[outk] = v
            elif k == "adelic-gens":
                cur[outk] = ast.literal_eval(v)
            else:
                cur[outk] = int(v)

    if cur:
        blocks.append(cur)

    return blocks